In [5]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm # Fortschrittsbalken

# 1. Modelle laden
print("Lade Übersetzer (DE -> EN)...")
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-de-en")

print("Lade Finanz-Sentiment (FinBERT)...")
finbert = pipeline("text-classification", model="ProsusAI/finbert")

# 2. Funktion
df_clean = pd.read_csv("../clean/portfolio_data_clean.csv")

def get_financial_sentiment(text):
    if pd.isna(text) or len(str(text)) < 10:
        return 0.0

    try:
        # A. Übersetzen (Nur die ersten 400 Zeichen reichen oft für den Kontext)
        # Längere Texte dauern zu lange zum Übersetzen
        text_snippet = text[:1000]
        translated = translator(text_snippet)[0]['translation_text']

        # B. FinBERT fragen
        result = finbert(translated)[0]
        label = result['label']
        score = result['score']

        # FinBERT Labels: 'positive', 'negative', 'neutral'
        if label == 'positive':
            return score
        elif label == 'negative':
            return -score
        else:
            return 0.0

    except Exception as e:
        print(f"Fehler: {e}") #
        return 0.0

# 3. Anwenden
tqdm.pandas() # Aktiviert progress_apply
df_clean['Sentiment_FinBERT'] = df_clean['Text_Clean'].progress_apply(get_financial_sentiment)
# 1. Schritt: Nullen entfernen (falls noch nicht geschehen)
df_no_zeros = df_clean[df_clean['Sentiment_FinBERT'] != 0.0].copy()

# 2. Schritt: Nur Zeilen mit einzigartigen Scores behalten
# drop_duplicates behält standardmäßig das erste Vorkommen und löscht den Rest
df_unique_scores = df_no_zeros.drop_duplicates(subset=['Sentiment_FinBERT'])
df_unique_scores.drop(columns=['Überschrift','Text_Clean'], inplace=True)
# 3. Anzeigen (Sortiert, damit man die Verteilung sieht)
print(f"Anzahl einzigartiger Scores (ohne 0): {len(df_unique_scores)}")
print(df_unique_scores[['Unternehmen', 'Datum', 'Sentiment_FinBERT']].sort_values(by='Sentiment_FinBERT'))
def fix_german_date(date_str):
    monate = {
        'Januar': '01', 'Februar': '02', 'März': '03', 'April': '04', 'Mai': '05', 'Juni': '06',
        'Juli': '07', 'August': '08', 'September': '09', 'Oktober': '10', 'November': '11', 'Dezember': '12'
    }

    for de, num in monate.items():
        if de in date_str:
            date_str = date_str.replace(de, num)
            break # Monat gefunden

    return date_str

df_unique_scores['Datum_Clean'] = df_unique_scores['Datum'].apply(fix_german_date)
df_unique_scores['Datum_Final'] = pd.to_datetime(df_unique_scores['Datum_Clean'], format='%d %m %Y')

print("Daten mit korrektem Datum:")
print(df_unique_scores.head())

Lade Übersetzer (DE -> EN)...


Device set to use mps:0


Lade Finanz-Sentiment (FinBERT)...


Device set to use mps:0
100%|██████████| 54/54 [04:00<00:00,  4.45s/it]

Anzahl einzigartiger Scores (ohne 0): 16
              Unternehmen            Datum  Sentiment_FinBERT
3              Bechtle AG    25 April 2025          -0.969149
46       SUSS MicroTec SE  27 Oktober 2025          -0.961266
7               CANCOM SE    29 April 2025          -0.911693
6               CANCOM SE     31 Juli 2025          -0.736025
35                 PNE AG      16 Mai 2025           0.568846
48       SUSS MicroTec SE   16 Januar 2025           0.610795
53                 1&1 AG  13 Februar 2025           0.766932
17  mobilezone holding AG   8 Oktober 2025           0.848288
52     United Internet AG  13 Februar 2025           0.872843
33     Emmi Management AG  26 Februar 2025           0.894219
29          Nemetschek SE     24 Juli 2025           0.901554
34     Emmi Management AG   24 Januar 2025           0.915200
37                 PNE AG  18 Februar 2025           0.928843
31              Nordex SE  27 Oktober 2025           0.949753
30          Nemetschek SE   2

In [9]:
df_unique_scores.to_csv('../clean/unique_scores.csv', index=False)

In [ ]:
# Einzeln exportieren
# Unternehmen = df_unique_scores['Unternehmen']
# for u in Unternehmen:
#    df_u = pd.DataFrame(['Unternehmen','Datum_Final','Sentiment_FinBERT'])
#    if u